<h1>Cylinder Volume Prediction</h1>
<p> As a chanllage we gernerated Training data with some radius and height using formula 
    $volume=\pi r^2 h$   where r is radius and h is height. Now we will train using either algo and will try to predict new data.

In [62]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.13.0-rc1


<h2>Read data</h2>

In [63]:
df = pd.read_csv('../input/CylinderVolumeDS.csv')

<H1>Add Cross Features

In [64]:
df=df[df['radius']>0]
df=df[df['height']>0]
df['rh'] = df['radius'] * df['height'] 
df['r2h'] = df['radius']*df['radius'] * df['height'] 
df['rh2'] = df['radius'] * df['height'] * df['height'] 
df['r3'] = df['radius'] * df['radius'] * df['radius'] 
df['h3'] = df['height'] * df['height'] * df['height'] 
    
    

<h1>Divide Dataset into Training and Validation

In [65]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df)) < 0.02
df_train = df[msk]

df_valid_n_test = df[~msk]

msk2 = np.random.rand(len(df_valid_n_test)) < 0.02
df_valid = df_valid_n_test[msk2]


df_test = df_valid_n_test[~msk2]
msk3 = np.random.rand(len(df_test)) < 0.02
df_test = df_test[msk3]


In [66]:
df_train.head()

,rowid,radius,height,volume,rh,r2h,rh2,r3,h3
102,103,1.5,0.5,3.534292,0.75,1.125,0.375,3.375,0.125
138,139,19.5,0.5,597.295303,9.75,190.125,4.875,7414.875,0.125
150,151,25.5,0.5,1021.410311,12.75,325.125,6.375,16581.375,0.125
198,199,49.5,0.5,3848.843700,24.75,1225.125,12.375,121287.375,0.125
207,208,4.0,1.0,50.265482,4.00,16.000,4.000,64.000,1.000


In [67]:
df_test.head()

,rowid,radius,height,volume,rh,r2h,rh2,r3,h3
132,133,16.5,0.5,427.649300,8.25,136.125,4.125,4492.125,0.125
142,143,21.5,0.5,726.100602,10.75,231.125,5.375,9938.375,0.125
158,159,29.5,0.5,1366.985503,14.75,435.125,7.375,25672.375,0.125
217,218,9.0,1.0,254.469005,9.00,81.000,9.000,729.000,1.000
233,234,17.0,1.0,907.920277,17.00,289.000,17.000,4913.000,1.000


<h2>Input Read data to Tensors</h2>

In [68]:
tf_feature_columns = [tf.feature_column.numeric_column('radius'),
                      tf.feature_column.numeric_column('height'),
                      tf.feature_column.numeric_column('rh'),
                     tf.feature_column.numeric_column('r2h'),
                     tf.feature_column.numeric_column('rh2'),
                     tf.feature_column.numeric_column('r3'),
                     tf.feature_column.numeric_column('h3')]
tf_input_train=tf.estimator.inputs.pandas_input_fn(x = df_train,
                                                   y = df_train['volume'],
                                                   batch_size=1000,
                                                   shuffle = True,
                                                  num_epochs=500)
tf_input_valid=tf.estimator.inputs.pandas_input_fn(x = df_valid,
                                                   y = df_valid['volume'],
                                                   shuffle = False)
tf_input_test=tf.estimator.inputs.pandas_input_fn(x = df_test,
                                                   y = None,    
                                                   shuffle = False)


<h2>Training - Linear Regressor</h2>

In [69]:
model = tf.estimator.DNNRegressor(hidden_units = [20,16, 8,3],
                                 feature_columns = tf_feature_columns)
#model = tf.estimator.LinearRegressor(feature_columns = tf_feature_columns, optimizer=tf.train.FtrlOptimizer(                          learning_rate=7000000,                                        l1_regularization_strength=0.00001                                    )                                    )
model.train(input_fn =tf_input_train)

<h2>Evaluating - Linear Regressor Model</h2>

In [70]:
metrics = model.evaluate(input_fn = tf_input_valid)
print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))

RMSE on dataset = 97293.6328125


<h2>Predicting - Linear Regressor Model</h2>

In [71]:
df_test.head()
predictions = model.predict(input_fn =tf_input_test)
t=[]
for items in predictions:
    t.append(items['predictions'])
    

df_test['PredictedVolume']=np.asarray(t )
df_test.head()

,rowid,radius,height,volume,rh,r2h,rh2,r3,h3,PredictedVolume
132,133,16.5,0.5,427.649300,8.25,136.125,4.125,4492.125,0.125,0.910748
142,143,21.5,0.5,726.100602,10.75,231.125,5.375,9938.375,0.125,0.910748
158,159,29.5,0.5,1366.985503,14.75,435.125,7.375,25672.375,0.125,0.910748
217,218,9.0,1.0,254.469005,9.00,81.000,9.000,729.000,1.000,0.910748
233,234,17.0,1.0,907.920277,17.00,289.000,17.000,4913.000,1.000,0.910748
